# Model a Golomb ruler problem by using decision optimization

Decision optimization is used for planning decisions, scheduling, and helping you make a data-based decision when large numbers of options are up for consideration.

This Python notebook shows you how to set up a decision optimization engine and create a constraint programming model that calculates and outputs a Golomb ruler.

In this notebook, you will model the Golomb ruler problem by using the <a href="https://rawgit.com/IBMDecisionOptimization/docplex-doc/master/docs/index.html" target="_blank" rel="noopener noreferrer">Decision Optimization for Python</a> API and solve it in IBM Decision Optimization on Cloud.

This notebook runs on Python 2.0 or Python 3.5 with Spark.

Table of contents:

-  [What is a Golomb ruler?](#golomb_ruler)
*  [How can decision optimization solve a problem](#decision_optimization)
*  [Model a Golomb ruler problem](#model_problem)
    *  [Step 1: Load library](#load_library)
    *  [Step 2: Set up the optimization engine](#set_up_prescriptive_engine)
    -  [Step 3: Model the input data](#model_data)
    -  [Step 4: Define the optimization model](#define_prescriptive_model)
        * [Define the decision variables](#define_variables)
        * [Express the business constraints](#express_constraints)
        * [Express the objective](#express_objective)
        * [Solve the model](#solve_model)
    *  [Step 5: Output the results](#output_results)
*  [Summary](#summary)
****

<a id="golomb_ruler"></a>
## What is a Golomb ruler?

A Golomb ruler is a sequence of non-negative integers such that the difference of two integer pairs in all sequences is distinct. Golomb rulers are used in many applications, including radio astronomy, information theory, and decision optimization to measure the distance between objects. 

For example:
    
-  In selecting radio frequencies to reduce the effects of intermodulation interference
-  In detecting and correcting errors in radio astronomy
-  In designing conference rooms, to maximize the number of possible configurations with a minimum of partitions
-  For positioning radio telescopes for optimal performance using the minimum number of telescopes on the least amount of land
-  For optimal positioning of x-ray sensors   

The following examples show Golomb rulers of the order 4, and length 6 and 11.

<br/>
<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/0/05/Golomb_Ruler-4.svg/220px-Golomb_Ruler-4.svg.png" align="left">
<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/5/52/Golomb_ruler_conference_room.svg/300px-Golomb_ruler_conference_room.svg.png" align="middle">


<a id="decision_optimization"></a>
## How can decision optimization solve a problem?

In decision optimization, actions are recommended based on the outcomes you desire, taking into account specific scenarios, resources, and knowledge of past and current events. Analyzing data to predict future outcomes and suggesting the optimal way to handle future outcomes can help you to make better decisions in uncertain situations and gain a strong competitive advantage.

 
Examples where decision otimization modeling can help:

-  Automating complex decisions and trade-offs to better manage limited resources
-  Taking advantage of a future opportunity or mitigating a future risk
-  Proactively updating recommendations based on changing events
-  Meeting operational goals, increasing customer loyalty, preventing threats and fraud, and optimizing business processes


<a id="model_problem"></a>
## Model a Golomb ruler problem

Constraint programming is a technology that is used in solving complex decision optimization problems. Constraint programming is a programming paradigm that allows you to express a problem by using:

-  The unknowns of the problem - the *variables*
-  The constraints or rules of the problem, the mathematical expressions that link the variables - the *constraints*
-  What you optimize - the *objective function*


All this information, plus some configuration parameters, is aggregated into a single object called a *model*. 

In this notebook, you will learn how to solve a Golomb ruler problem with IBM Constraint Programming Optimizer, using the *DOcplex* Python modeling API.

<a id="load_library"></a>
### Step 1: Load library

Run the following code to install the Decision Optimization CPLEX Modeling library.  The *DOcplex* library contains the two modeling packages, Mathematical Programming and Constraint Programming.

In [ ]:
!pip install docplex
import sys
import docplex.cp

Note that the global `docplex` package contains the subpackage `docplex.mp` that is dedicated to Mathematical Programming, another branch of optimization.

<a id="set_up_prescriptive_engine"></a>
### Step 2: Set up the optimization engine

DOcplexcloud includes a web service, DropSolve, where you can drag and drop problem files, download results, and manage your jobs in the queue.

To use the DropSolve service:

-  Register for the [DOcplexcloud 30 days free trial](https://developer.ibm.com/docloud).
-  Get your [access credentials (the base URL and access key)](http://developer.ibm.com/docloud/docs/api-key/).

Add the URL and API key to the following cell and run it:

In [2]:
# Initialize IBM Decision Optimization credentials
SVC_URL = " " #ENTER URL HERE
SVC_KEY = " " # ENTER KEY HERE

Now, import the required modeling functions that are provided by the *docplex.cp* package:

In [3]:
# Import Constraint Programming modelization functions
from docplex.cp.model import *

<a id="model_data"></a>
### Step 3: Model the input data

The first thing to define is the input data to the model. In the case of the Golomb ruler problem, there is only one input,  which is the order of the ruler, that is, the number of marks on the ruler: 


In [4]:
# Define required number of marks on the ruler
ORDER = 7

<a id="define_prescriptive_model"></a>
### Step 4: Define the optimization model

The optimization model is represented by a container that is filled with the different model elements, in this case, the variables, constraints, and the objective function. The container is a Python model object.

Run the following cell to create a Python model object:

In [5]:
# Create model object
mdl = CpoModel()

<a id="define_variables"></a>
#### Define the decision variables

Now define the variables of the model. As the expected problem result is the list of mark positions, the simplest choice is to create one integer variable to represent the position of each mark on the ruler.

Each variable has a set of possible values called its *domain*. To reduce the search space, it is important to reduce this domain as far as possible. For the domain, you can estimate that the maximum distance between two adjacent marks is the order of the ruler minus one. Then the maximal position of a mark is `(ORDER - 1)²`. Each variable domain is then limited to an interval `[0..(ORDER - 1)²]`.

You can define a list of integer variables by using the `integer_var_list()`. Run the following cell to define one variable for each mark:

In [6]:
# Create array of variables corresponding to ruler marks
marks = integer_var_list(ORDER, 0, (ORDER - 1) ** 2, "M")

<a id="express_constraints"></a>
#### Express the business constraints

For a Golomb ruler, you need to express that all possible distances between two marks are different. To do this, create an array that contains all these distances:

In [7]:
# Create an array with all distances between all marks
dist = [marks[i] - marks[j] for i in range(1, ORDER) for j in range(0, i)]

In the computation, you used the `-` operator to express the difference between variables. Using this operator might appear strange because the variables are not instanciated at that time. The Python operator is overloaded to construct a constraint programming expression instead of attempting to compute the arithmetic difference. All of the standard Python operators `(<, >, <=, >=, ==, !=, +, -, /, *, &, |, //, **,...)` can be used for operations between constraint programming objects. 

To force the distances to be different, use the `all_diff()` constraint:

In [8]:
# Force all distances to be different
mdl.add(all_diff(dist))

The `mdl.add(...)` function is necessary to express that the constraint is to be added to the model.

##### Remove symmetries

Although the `all_diff()` constraint is enough to solve the model, it does not differentiate between all possible permutations of the different mark positions that could be solutions to the problem, for example, `0-1-4-6, 4-6-1-0, 6-0-1-4`, and so on. Because  there is a factorial magnitude of such permutations, the search space is drastically reduced by removing them.

To reduce the permutations, run the next cell to force an order between marks, for example, the order of their index:

In [9]:
# Avoid symmetric solutions by ordering marks
for i in range(1, ORDER):
    mdl.add(marks[i] > marks[i - 1])

You know that the first mark of the ruler is zero:

In [10]:
# Force first mark position to zero
mdl.add(marks[0] == 0)

##### Avoid mirror solutions

Each optimal solution has a mirror, with all mark distances in the reverse order, for example, `0-1-4-6` and `0-2-5-6`. 
The following constraint ensures that the reverse order won't occur: 

In [11]:
# Avoid mirror solution
mdl.add((marks[1] - marks[0]) < (marks[ORDER - 1] - marks[ORDER - 2]))

<a id="express_objective"></a>
#### Express the objective

You don't only want the marks to be as close together as possible. More specifically, you want to find the shortest possible ruler for a given number of marks. Such a ruler is called optimal.

As the marks are ordered, you can express this by minimizing the position of the last mark:

In [12]:
# Minimize ruler size
mdl.add(minimize(marks[ORDER - 1]))

If the marks are not ordered, you would run the following code instead:<br>
<code>   mdl.add(minimize(max(marks)))</code><br>

<a id="solve_model"></a>
#### Solve the model 

You have finished defining the model. Now, it's time to solve it by using the Decision Optimization on Cloud DropSolve service.

Run the following cell to solve the model:

In [13]:
# Solve the model
print("Solving model....")
msol = mdl.solve(url=SVC_URL, key=SVC_KEY, TimeLimit=10)

Solving model....


<a id="output_results"></a>
### Step 5: Output the results

The quickest way to output what was found by the DropSolve service is to call the method `print_solution()` as follows:

In [14]:
# Print solution
print("Solution: ")
msol.print_solution()

Solution: 
-------------------------------------------------------------------------------
Model constraints: 9, variables: integer: 7, interval: 0, sequence: 0
Solve status: Optimal
Search status: SearchCompleted, stop cause: SearchHasNotBeenStopped
Solve time: 5.03 sec
-------------------------------------------------------------------------------
Objective values: (25,)
          bounds: (25,)
          gaps: (0.0,)
M_0: 0
M_1: 1
M_2: 11
M_3: 16
M_4: 19
M_5: 23
M_6: 25


However, this output is very generic and simply prints the value of all the model variables, the objective value, and some other solving information.

You can generate more specific output by writing code that accesses various elements of the solution: 

In [15]:
# Print solution
from sys import stdout
if msol:
    # Print found solution
    stdout.write("Solution: " + msol.get_solve_status() + "\n")
    stdout.write("Position of ruler marks: ")
    for v in marks:
        stdout.write(" " + str(msol[v]))
    stdout.write("\n")
    stdout.write("Solve time: " + str(round(msol.get_solve_time(), 2)) + "s\n")
else:
    # No solution found
    stdout.write("No solution found. Search status: " + msol.get_solve_status() + "\n")

Solution: Optimal
Position of ruler marks:  0 1 11 16 19 23 25
Solve time: 5.03s


Another possibility, for example, is  to simulate a real ruler using characters, as follows:

In [16]:
# Print solution as a ruler
if msol:
    stdout.write("Ruler: +")
    for i in range(1, ORDER):
        stdout.write('-' * (msol[marks[i]] - msol[marks[i - 1]] - 1) + '+')
    stdout.write("\n")

Ruler: ++---------+----+--+---+-+


<a id="summary"></a>
## Summary

You learned how to set up and use IBM Decision Optimization CPLEX Modeling for Python to formulate a constraint programming model for a Golomb ruler problem and how to solve the problem with IBM Decision Optimization on Cloud.

### References


* <a href="https://rawgit.com/IBMDecisionOptimization/docplex-doc/master/docs/index.html" target="_blank" rel="noopener noreferrer">The latest PyPi library</a>
* <a href="https://github.com/IBMDecisionOptimization/docplex-examples" target="_blank" rel="noopener noreferrer">Modeling examples for download</a>
* <a href="https://rawgit.com/IBMDecisionOptimization/docplex-doc/master/docs/index.html" target="_blank" rel="noopener noreferrer">CPLEX Modeling for Python documentation</a>
* <a href="https://developer.ibm.com/docloud/" target="_blank" rel="noopener noreferrer">Decision Optimization on Cloud</a>
* Need help with DOcplex or want to report a bug? Go <a href="https://developer.ibm.com/answers/smartspace/docloud" target="_blank" rel="noopener noreferrer">here</a>
* Contact us at `dofeedback@wwpdl.vnet.ibm.com`

### Author

**Olivier Oudot**, Ph.D. in Logical Programming and IBM Master Inventor, joined the development team of the IBM Constraint Programming Optimizer in 2010 after a career of software development in military, space, and telecommunications areas. He worked on performance measurements for the CPO solver before developing its Python interface, part of `docplex` initiative (docplex.cp).


Copyright © 2017, 2018 IBM. This notebook and its source code are released under the terms of the MIT License.

<div style="background:#F5F7FA; height:110px; padding: 2em; font-size:14px;">
<span style="font-size:18px;color:#152935;">Love this notebook? </span>
<span style="font-size:15px;color:#152935;float:right;margin-right:40px;">Don't have an account yet?</span><br>
<span style="color:#5A6872;">Share it with your colleagues and help them discover the power of Watson Studio!</span>
<span style="border: 1px solid #3d70b2;padding:8px;float:right;margin-right:40px; color:#3d70b2;"><a href="https://ibm.co/wsnotebooks" target="_blank" style="color: #3d70b2;text-decoration: none;">Sign Up</a></span><br>
</div>